<a href="https://colab.research.google.com/github/KayvanShah1/usc-dsci553-data-mining-sp24/blob/main/assignment-1/notebooks/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [1]:
!pip install pyspark
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=010f2a8c4a67e6a74f9a5202451cdc2458fab6c2969ad7c761c51ef2c150ef7b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.0 MB/s eta 0:00:00
time: 396 µs (started: 2024-02-09 01:21:29 +00:00)


In [1]:
%%bash
java --version

openjdk 11.0.21 2023-10-17
OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)


# Imports

In [22]:
import os
import sys
import json
from pyspark import SparkContext

from pprint import pprint

# Task 1

In [43]:
import sys
import json
from datetime import datetime

from pyspark import SparkContext


def task1(review_file_path, output_file_path):
    spark = SparkContext(appName="Yelp Data Exploration (Task 1)").getOrCreate()

    review_rdd = spark.textFile(review_file_path).map(json.loads).cache()

    result = {
        "n_review": 0,
        "n_review_2018": 0,
        "n_user": 0,
        "top10_user": [],
        "n_business": 0,
        "top10_business": []
    }

    try:
        result["n_review"] = review_rdd.count()

        result["n_review_2018"] = review_rdd.filter(
            lambda x: datetime.strptime(x["date"], "%Y-%m-%d %H:%M:%S").year==2018
        ).count()

        result["n_user"] = review_rdd.map(lambda x: x["user_id"]).distinct().count()

        result["top10_user"] = (
            review_rdd.map(lambda x: (x["user_id"], 1))
            .reduceByKey(lambda a, b: a + b)
            .sortBy(lambda x: (-x[1], x[0]))
            .take(10)
        )

        result["n_business"] = review_rdd.map(lambda x: x["business_id"]).distinct().count()

        result["top10_business"] = (
            review_rdd.map(lambda x: (x["business_id"], 1))
            .reduceByKey(lambda a, b: a + b)
            .sortBy(lambda x: (-x[1], x[0]))
            .take(10)
        )

        with open(output_file_path, "w") as f:
            json.dump(result, f, indent=4)

        pprint(result)

    finally:
        spark.stop()


# if __name__ == "__main__":
#     if len(sys.argv) != 3:
#         print("Usage: spark-submit --executor-memory 4G --driver-memory 4G task1.py <review_file_path> <output_file_path>")
#         sys.exit(1)
#     review_filepath = sys.argv[1]
#     output_filepath = sys.argv[2]
#     task1(review_filepath, output_filepath)

task1("test_reviews.jsonl", "python_task1.json")

{'n_business': 30,
 'n_review': 30,
 'n_review_2018': 3,
 'n_user': 30,
 'top10_business': [('3fw2X5bZYeW9xCz_zGhOHg', 1),
                    ('6lj2BJ4tJeu7db5asGHQ4w', 1),
                    ('8mIrX_LrOnAqWsB5JrOojQ', 1),
                    ('9nTF596jDvBBia2EXXiOOg', 1),
                    ('AakkkTuGZA2KBodKi2_u8A', 1),
                    ('FQ1wBQb3aNeRMThSQEV0Qg', 1),
                    ('FxLfqxdYPA6Z85PFKaqLrg', 1),
                    ('Gyrez6K8f1AyR7dzW9fvAw', 1),
                    ('I4Nr-MVc26qWr08-S3Q1ow', 1),
                    ('LUN6swQYa4xJKaM_UEUOEw', 1)],
 'top10_user': [('-mA3-1mN4JIEkqOtdbNXCQ', 1),
                ('2mxBNBeFrgDszqGS5tdEHA', 1),
                ('3CJUJILq7CLHk_9OrvpvQg', 1),
                ('5JVY32_bmTBfIGpCCsnAfw', 1),
                ('6Fz_nus_OG4gar721OKgZA', 1),
                ('86J5DwcFk4f4In1Vxe2TvA', 1),
                ('8NwU4TRsD3S6gIfBqFzDMQ', 1),
                ('DzZ7piLBF-WsJxqosfJgtA', 1),
                ('FIk4lQQu1eTe2EpzQ4xhBA'

In [44]:
%%writefile task1.py
import json
import sys
from datetime import datetime

from pyspark import SparkContext


def task1(review_file_path, output_file_path):
    spark = SparkContext(appName="Yelp Data Exploration (Task 1)").getOrCreate()

    review_rdd = spark.textFile(review_file_path).map(json.loads).cache()

    result = {
        "n_review": 0,
        "n_review_2018": 0,
        "n_user": 0,
        "top10_user": [],
        "n_business": 0,
        "top10_business": [],
    }

    try:
        result["n_review"] = review_rdd.count()

        result["n_review_2018"] = review_rdd.filter(
            lambda x: datetime.strptime(x["date"], "%Y-%m-%d %H:%M:%S").year == 2018
        ).count()

        result["n_user"] = review_rdd.map(lambda x: x["user_id"]).distinct().count()

        result["top10_user"] = (
            review_rdd.map(lambda x: (x["user_id"], 1))
            .reduceByKey(lambda a, b: a + b)
            .sortBy(lambda x: (-x[1], x[0]), ascending=True)
            .take(10)
        )

        result["n_business"] = review_rdd.map(lambda x: x["business_id"]).distinct().count()

        result["top10_business"] = (
            review_rdd.map(lambda x: (x["business_id"], 1))
            .reduceByKey(lambda a, b: a + b)
            .sortBy(lambda x: (-x[1], x[0]), ascending=True)
            .take(10)
        )

        with open(output_file_path, "w") as f:
            json.dump(result, f, indent=4)

    finally:
        spark.stop()


if __name__ == "__main__":
    if len(sys.argv) != 3:
        print(
            "Usage: spark-submit --executor-memory 4G --driver-memory 4G "
            "task1.py <review_file_path> <output_file_path>"
            " --conf spark.driver.extraJavaOptions=-Dlog4j.configuration=log4j.properties"
        )
        sys.exit(1)

    review_filepath = sys.argv[1]
    output_filepath = sys.argv[2]
    task1(review_filepath, output_filepath)


Overwriting task1.py


In [45]:
!spark-submit --executor-memory 4G --driver-memory 4G task1.py test_reviews.jsonl python_task1.json --conf spark.driver.extraJavaOptions=-Dlog4j.configuration=log4j.properties

Usage: spark-submit --executor-memory 4G --driver-memory 4G task1.py <review_file_path> <output_file_path> --conf spark.driver.extraJavaOptions=-Dlog4j.configuration=log4j.properties
24/02/09 03:55:06 INFO ShutdownHookManager: Shutdown hook called
24/02/09 03:55:06 INFO ShutdownHookManager: Deleting directory /tmp/spark-15424baa-744a-4a31-8be1-4e9c87aa17ca


# Task 2

In [134]:
import json
import sys
from time import time

from pyspark import SparkContext


def custom_partition_func(record, n_partition):
    return hash(record[0]) % n_partition


def task2(review_file_path, output_file_path, n_partition):
    spark = SparkContext(appName="Yelp Data Exploration (Task 2)").getOrCreate()

    review_rdd = spark.textFile(review_file_path).map(json.loads).cache()

    result = {
        "default": {"n_partition": 0, "n_items": [], "exe_time": 0},
        "customized": {"n_partition": 0, "n_items": [], "exe_time": 0},
    }

    try:
        # Default partition function
        start = time()
        _ = (
            review_rdd.map(lambda x: (x["business_id"], 1))
            .reduceByKey(lambda x, y: x + y)
            .sortBy(lambda x: (-x[1], x[0]), ascending=True)
            .take(10)
        )
        end = time()

        result["default"]["n_partition"] = review_rdd.getNumPartitions()

        result["default"]["n_items"] = review_rdd.glom().map(len).collect()

        result["default"]["exe_time"] = end - start

        # Customized partition function
        start = time()
        partitioned_rdd = (
            review_rdd.map(lambda x: (x["business_id"], 1))
            .partitionBy(n_partition, lambda record: custom_partition_func(record, n_partition))
            .cache()
        )

        _ = (
            partitioned_rdd.reduceByKey(lambda x, y: x + y)
            .sortBy(lambda x: (-x[1], x[0]), ascending=True)
            .take(10)
        )
        end = time()

        result["customized"]["n_partition"] = partitioned_rdd.getNumPartitions()

        result["customized"]["n_items"] = partitioned_rdd.glom().map(len).collect()

        result["customized"]["exe_time"] = end - start

        pprint(result)

        with open(output_file_path, "w") as f:
            json.dump(result, f, indent=4)

    finally:
        spark.stop()


task2("review.jsonl", "python_task2.json", 3)

{'customized': {'exe_time': 2.0041656494140625,
                'n_items': [393, 419, 419],
                'n_partition': 3},
 'default': {'exe_time': 2.1744067668914795,
             'n_items': [484, 747],
             'n_partition': 2}}


In [123]:
%%writefile task2.py
import json
import sys
from time import time

from pyspark import SparkContext


def custom_partition_func(record):
    return hash(record[0]) % n_partition


def task2(review_file_path, output_file_path, n_partition):
    spark = SparkContext(appName="Yelp Data Exploration (Task 2)").getOrCreate()

    review_rdd = spark.textFile(review_file_path).map(json.loads).cache()

    result = {
        "default": {"n_partition": 0, "n_items": [], "exe_time": 0},
        "customized": {"n_partition": 0, "n_items": [], "exe_time": 0},
    }

    try:
        # Default partition function
        start = time()
        _ = (
            review_rdd.map(lambda x: (x["business_id"], 1))
            .reduceByKey(lambda x, y: x + y)
            .sortBy(lambda x: (-x[1], x[0]), ascending=True)
            .take(10)
        )
        end = time()

        result["default"]["n_partition"] = review_rdd.getNumPartitions()

        result["default"]["n_items"] = review_rdd.glom().map(len).collect()

        result["default"]["exe_time"] = end - start

        # Customized partition function
        start = time()
        partitioned_rdd = (
            review_rdd.map(lambda x: (x["business_id"], 1))
            .partitionBy(n_partition, custom_partition_func)
            .cache()
        )

        _ = (
            partitioned_rdd.reduceByKey(lambda x, y: x + y)
            .sortBy(lambda x: (-x[1], x[0]), ascending=True)
            .take(10)
        )
        end = time()

        result["customized"]["n_partition"] = partitioned_rdd.getNumPartitions()

        result["customized"]["n_items"] = partitioned_rdd.glom().map(len).collect()

        result["customized"]["exe_time"] = end - start

        with open(output_file_path, "w") as f:
            json.dump(result, f, indent=4)

    finally:
        spark.stop()


if __name__ == "__main__":
    if len(sys.argv) != 4:
        print(
            "Usage: spark-submit --executor-memory 4G --driver-memory 4G"
            " task2.py <review_file_path> <output_file_path> <n_partitions>"
        )
        sys.exit(1)

    review_file_path = sys.argv[1]
    output_file_path = sys.argv[2]
    n_partition = int(sys.argv[3])

    task2(review_file_path, output_file_path, n_partition)


Writing task2.py


In [130]:
!spark-submit --executor-memory 4G --driver-memory 4G task2.py review.jsonl python_task2.json 2 --conf spark.driver.extraJavaOptions=-Dlog4j.configuration=log4j.properties

Usage: spark-submit --executor-memory 4G --driver-memory 4G task2.py <review_file_path> <output_file_path> <n_partitions>
24/02/09 07:07:18 INFO ShutdownHookManager: Shutdown hook called
24/02/09 07:07:18 INFO ShutdownHookManager: Deleting directory /tmp/spark-5f1b5ecd-20d4-4abd-84e9-9b0af863b234


# Task 3

# END